In [1]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    confusion_matrix, classification_report

from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve

from sklearn.model_selection import learning_curve

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.inspection import permutation_importance
#import warnings
#warnings.simplefilter(action="ignore")

In [3]:
immo = pd.read_csv("immo.csv")
immo_with_imput = pd.read_csv("immo_with_imput.csv")
immo_with_outliers = pd.read_csv("immo_with_outliers.csv")
immo_with_outliers_and_imput = pd.read_csv("immo_with_outliers_and_imput.csv")

In [4]:
# define X, y
y = immo["ocean_proximity_number"]
X = immo.drop("ocean_proximity_number", axis=1)

In [5]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.43564292201894383

In [6]:
# define X, y
y = immo_with_imput["ocean_proximity_number"]
X = immo_with_imput.drop("ocean_proximity_number", axis=1)

In [7]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.43564292201894383

In [8]:
# define X, y
y = immo_with_outliers["ocean_proximity_number"]
X = immo_with_outliers.drop("ocean_proximity_number", axis=1)

In [9]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.4428294573643411

In [10]:
# define X, y
y = immo_with_outliers_and_imput["ocean_proximity_number"]
X = immo_with_outliers_and_imput.drop("ocean_proximity_number", axis=1)

In [11]:
# instantiate Dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X, y)

# calculate the score
dummy_clf.score(X, y)

0.4428294573643411

---We can see a little improvement with the dataset : immo_with_outliers_and_imput and immo_with_outliers---

---for now i'll keep the outliers---